# Who is Working on this Project

- Tyler Arista
  - Responsibilties
- Lydia Powers
  - Responsbilities
- Chang Liu
  - Responsibilities

# Vision

Our goal for this project is to see how machine learning can help improve the quality of academic essays/papers. We start by choosing an exisitng essay and using an machine learning model to create an intial outlien of it. This outline is then check and corrected by a person to make sure it is accurate and relevant. After that, we use the corrected outlien to help the ML model write a neew version of the essay.

To check if our method works well, we compare the original essay written by a human with the new one written by the model. We look closely to each word(token) in both essays and calculate how well each word fits. If the model's word fits better, we replace the orignal word with teh AI's choice. This process lets us try out different ways of giving information to the model, like using just the original essay, the corrected outline, both together, the outline made by the model, or even no context at all.

Our aim is to find out which way of giving information lets the model do its best job in terms of making fewer mistakes and keeping the text clear and well-organized. If our project is successful, it would show that the AI can help make the orignal essay better, resulting in a final piece that is clear, well-written,a nd improved with teh help of the model

# Background

For this project, the primary data comprise sof essays form previous college courses written by the group members. This dataset was chosen because it provides authentic academci writing samples that reflect typical structures, themes, and complexity expected in college-level essays. These essays serve as an ideal basis for evaluating the effectiveness of machine learning models in generating and refining academic text, as they represent real-world examples of content that students are expected to produce. We have confidence in the primary dataset because of several factors:
- Relevance: Essays from previous courses are directly relevant to the project's aim, which is to enhance the quality and coherence of academci essays using AI
- Diversity: These essays cover a range of subjects and writing styles, providng a robust base for testing the AI's adaptibility and accuracy in generating and refining text
- Authenticity: Using actual student essays ensures the project results are applicable in real-world educational settings, enhacing the practical value of the research

What technologies are you using?
- Pytorch
- Transformers Library from Hugging Face/GEMMA
- Pandas & Numpy

# Implementation
What prior code can you build on?
Your final report will summarize your implementation and, if appropriate, how it extends the work you’ve reverenced.

# Prompt, Outline/Essay -(Put Through Model)-> Genereated Outline --> Human Edited Outline of G.O. --> Revised Essay Per Token

In [2]:
import torch, os
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
# Work around a bug in the version of PyTorch and GPU hardware curretnly on Kaggle. On other hardware, removing these lines may lead to a speed-up.
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Load the model
USE_INSTRUCTION_TUNED = True # we'll switch this to True partway through the lab
if USE_INSTRUCTION_TUNED:
    model_name = '/kaggle/input/gemma/transformers/1.1-2b-it/1'
    if not os.path.exists(model_name):
        print("Warning: loading model weights from the Internet. This might take a bit of extra time.")
        model_name = "google/gemma-1.1-2b-it"
else:
    model_name = "/kaggle/input/gemma/transformers/2b/2"
    if not os.path.exists(model_name):
        print("Warning: loading model weights from the Internet. This might take a bit of extra time.")
        model_name = "google/gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.bfloat16)
streamer = TextStreamer(tokenizer)
# Silence a warning.
tokenizer.decode([tokenizer.eos_token_id]);

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-04-28 19:17:11.237040: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 19:17:11.237140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 19:17:11.380565: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [39]:
# Check where the whole model is loaded and what data type it's using.
model.device, model.dtype

(device(type='cuda', index=0), torch.bfloat16)

In [40]:
# Check where parameters are loaded. If this is anything other than {'': 0}
# then probably some parts of the model got offloaded onto CPU and so will run slow.
model.hf_device_map

{'': 0}

# Tyler's Essay Components
### Prompt(Tyler)

In [113]:
tyler_essay_prompt = '''In Homegoing, Pachinko, and Stories We Tell, the authors/filmmaker choose deliberate artistic strategies to present the histories they narrate. Discuss how the literary/filmic choices of Gyasi, Lee, and Polley are part of their overall theory about how the past should be constructed. In other words, how does the way in which they present their stories intersect with what they are trying to say in their stories? Are there commonalities in their aims? If so, what are they? Are there critical differences?'''

### Outline given by the User(Tyler)

In [114]:
tyler_outline_first_paragraph = '''
Thesis: Authors and filmmaker use multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, demonstrating their theories on how history should be constructed.
Brief introduction of works and creators: Discusses Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.
'''

In [115]:
tyler_outline_second_paragraph = '''
Main idea: Personal decisions directly shape characters' identities and futures in all three works, highlighting the authors' and filmmaker's focus on the impact of individual agency within broader historical and social contexts.
Explanation: This exploration of personal choice aligns with the creators' views that history is not merely a series of events but a complex tapestry woven from individual actions and their consequences.
Example from Pachinko: Sunja's pivotal decision to engage with Hansu, and its ramifications, showcase how personal mistakes and moral dilemmas are central to character development and plot progression.
Example from Homegoing: The character H’s experience with forced labor and subsequent physical development illustrates how personal endurance and adaptation to circumstances reflect broader historical forces like slavery and institutional racism.
Example from Stories We Tell: Harry’s narrative and his one-sided love affair demonstrate how personal perceptions can deeply influence one’s identity and the stories they choose to tell or believe.
'''

In [116]:
tyler_outline_third_paragraph = '''
Main idea: Characters’ identities and life paths are significantly influenced by the actions and statuses of those around them and their ancestors, emphasizing the interconnectedness of personal histories within larger societal narratives.
Explanation: This theme underscores the authors' and filmmaker's perspective that individual lives are not isolated but are deeply affected by the historical and relational contexts in which they exist, echoing a broader theory that history is constructed collectively rather than singularly.
Example from Pachinko: Isak’s altruistic decision to marry Sunja provides a stark contrast to her initial dilemma, showing how benevolent actions from others can redirect an individual’s life trajectory dramatically.
Example from Homegoing: The legacy of slavery, as seen through Esi and her descendant Ness, highlights how ancestral histories cast long shadows over the lives of future generations, shaping identities and opportunities long after the original events have passed.
Example from Stories We Tell: The revelation of Sarah’s true paternity and Diane’s decisions regarding her upbringing illustrate the profound impact parental choices have on children’s identities and their understanding of family narratives.
'''

### Human Written Essay(Tyler)

In [117]:
tyler_essay_first_paragraph = '''Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. '''

In [118]:
tyler_essay_second_paragraph = '''People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we mainly see the story through the viewpoint of Sunja, but we will occasionally get to look through or see the thoughts of the people close to her. In the novel, it states, “If he did not marry her, she was a common slut who would be disgraced forever. The child would be another no-name bastard. Her mother’s boardinghouse would be contaminated by her shame”(Lee 49). Sunja made a big mistake by being with Hansu, and she paid the price. She was now pregnant, and the father of the child won’t be able to marry her because he is already married. In her society, bearing a child without a father will lead to having the mother, in this case, Sunja, be disowned. It could also lead to her family and her child being disowned as well. Looking through the lens of Sunja, we could see how devastated she was and the impact her decisions could make on her in the future. Also, in the novel, Homegoing by Yaa Gyasi, we see through many different lenses in many different generations. In the chapter about H, it states, “The boss man was called Mr. John. He asked to take off his shirt. He inspected the muscles on his back and his arms and whistled. ‘Any man what can spend ten years working at Rock Slope and live to tell about it’s worth watching”(Gyasi 169). When H was sent to me while he was arrested, he might not have had an option on whether or not he wanted to work, but it made him better in the long run. He had gotten physically strong, and when he had gotten released, he was able to find a mining job that paid him. The situation he was put in shaped the person that H was. He became a hard worker and he made sure he did his job. Those were some of the things that he learned while he mined for the jail, but he was able to apply those lessons to the real world and shaped him to be somebody that was hirable. Another example of identities being defined by self choices is in the documentary, Stories We Tell, produced by Sarah Polley. When Harry met with Diane when she traveled to Montreal for a play, Harry fell in love with her. He had his own one-sided story, where he thought he was going to be with Diane for the rest of his life. This ultimately shaped his identity for the future. At the end of the documentary, Sarah asked Harry whether he liked that many people are sharing their sides of the story or if he disliked it. He shared that he did not like how many people shared their sides because he thought it was only his story to tell. He got wrapped up in the sense that Diane only loved him, and that changed the way he viewed his story.'''

In [119]:
tyler_essay_third_paragraph = '''While some believe that their identity is solely based on their choices, people’s identities can be altered by the people around them and by their ancestors. When we start looking through the other lenses of people in the same situation, we can see that many times the expected outcome is different from reality. In the novel, Pachinko, it states, “‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'”(Lee 74). Isak knew the dilemma Sunja was in and he knew this was something that shouldn’t be taken lightly. After coming up with the idea of marrying Sunja and giving the child his last name, he had lifted the burden off Sunja and her child’s back. They would no longer have to bear the weight of being dishonored by society and her family. Having people who can have a different perspective of the matter, can allow one to alter the course of somebody. Then, in the novel, Homegoing, we were able to witness the identities being altered because of their past relatives. In the novel, it states, “Every day, Ness picked cotton under the punishing eye of the southern sun. She had been at Thomas Allan Stockham’s Alabama plantation for three months”(Gyasi 70). Ness was a descendant of Esi in this story. Esi was a part of the slave trade system and her life was very rough. Esi went through an enormous amount of abuse and unbearable situations. Since being in that slave system, it had affected the rest of her family tree. Ness is the daughter of Esi, and so, Ness had to bear the identity of being a slave just like her mother. She had to be a slave on a plantation and no choice she made would be able to change that in this system. Her identity at the time was determined by her mother and how she was a part of the system. Lastly, in the documentary, Stories We Tell, Sarah’s identity was shaped by the choices her mother made when she was alive. Sarah never knew that Michael was not her biological father until she grew up. Sarah was able to form a strong bond with Michael, and that was because Diane decided to have Michael take care of her instead of Harry. Once Sarah found out that Harry was her father, Sarah was still able to keep the close relationship with Michael even after finding out the truth. Sarah was able to keep her identity from the one she formed with Michael, and that was all due to Diane.'''

# Lydia's Essay Components
### Prompt(Lydia)

In [80]:
lydia_prompt = ''''''

### Outline given by the User(Lydia)

In [81]:
lydia_outline_first_paragraph = ''''''

### Human Written Essay(Lydia)

In [82]:
lydia_essay_first_paragraph = ''''''

# Chang's Essay Components
### Prompt(Chang)

In [84]:
chang_prompt = '''Evaluate the ethical implications of architectural design in public spaces, using Tiananmen Square as a case study.'''

### Outline given by the User(Chang)

In [85]:
chang_outline_first_paragraph = '''Overview of Tiananmen Square's significance and thesis statement.'''

In [ ]:
chang_outline_second_paragraph = '''Explanation of Tiananmen Square's design to serve people and state leaders. Versatility for various events and ceremonies.'''

In [ ]:
chang_outline_third_paragraph = '''Description of nearby crucial buildings and their roles. Analysis of how these buildings enhance Tiananmen Square's importance. Detailing the purposes of Tiananmen Square's three entrances. Illustration of the diverse events facilitated through each entrance.'''

In [ ]:
chang_outline_forth_paragraph = '''Recap of Tiananmen Square's ethical architecture. Emphasis on its enduring significance in Chinese culture and history'''

### Human Written Essay(Chang)

In [86]:
chang_essay_first_paragraph = ''' Tiananmen Square is mostly ethical because it was designed to serve the people and the state leaders in many ways, it was surrounded by important buildings, and it had three entrances each serving in a different way. '''

In [ ]:
chang_essay_second_paragraph = '''Tiananmen Square can be used as a place for reviewing troops, visiting, and all kinds of parades. In most independence days and army days, China will have a big reviewing troops ceremony going on in Tiananmen Square. Soldiers and all kinds of weaponed cars will march through Tiananmen Square. It shows to the world that China is a powerful country. Tiananmen Square is also a popular visit place for foreigners. There were millions of people visiting every year. It shows both the modern and ancient side of China. A lot of people might not know but Tiananmen Square also has a lot of parades going on every year. Especially during the spring festival, there will be lots of decorated cars and dancing people around Tiananmen Square to celebrate the coming of the new year. '''

In [ ]:
chang_essay_third_paragraph = '''Tiananmen Square is located in the center of the city of Beijing which is also surrounded by lots of crucial buildings like the Chinese museum, the Great Hall of People, and the Forbidden City. On the east side of Tiananmen Square stands the Chinese museum. The Chinese museum is the largest comprehensive museum in China. It contains some of China’s most valuable national treasures. To the west is the Great Hall of People where the state leaders and the president hold all the important meetings. Lastly to the north was the Forbidden City. The Forbidden City is one of the biggest wooden structures in the world. It was built 600 years ago and the royal family of ancient China lived there for hundreds of years. Tiananmen Square was surrounded by all of those important architects. '''

In [ ]:
chang_essay_forth_paragraph = '''It can be used in ways, it is surrounded by some of the most significant buildings in China, and even its three doors have different meanings. All of these reasons made Tiananmen Square an ethical building. '''

# Compute loss of each token(word by word) of the Original Human Generated Essay

In [126]:
import torch
import pandas as pd

def analyze_text(essay):
    # Tokenize the essay for comparison
    essay_ids = tokenizer.encode(essay, return_tensors='pt').to(model.device)
    
    # Generate logits for the essay input
    with torch.no_grad():
        outputs = model(essay_ids)
        logits = outputs.logits
    
    # Analyze tokens for loss
    spans = []
    highest_loss = float('-inf')
    softmax = torch.nn.Softmax(dim=-1)
    essay_tokens = tokenizer.convert_ids_to_tokens(essay_ids[0])
    
    for i in range(1, essay_ids.size(1)):  # Start from 1 to skip the first token (usually [CLS] or similar)
        probs = softmax(logits[0, i - 1])
        token_loss = -torch.log(probs[essay_ids[0, i]]).item()
        most_likely_token_id = torch.argmax(probs).item()
        token = essay_tokens[i]  # Adjust index for essay tokens
        most_likely_token = tokenizer.decode([most_likely_token_id])
        
        spans.append({
            'original_token': token,
            'token_loss': token_loss,
            'most_likely_token': most_likely_token,
            'loss_ratio': None  # To be calculated later
        })
        
        if token_loss > highest_loss:
            highest_loss = token_loss

    # Normalize loss ratios
    for span in spans:
        span['loss_ratio'] = span['token_loss'] / highest_loss if highest_loss != 0 else 0

    df = pd.DataFrame(spans)
    return df


### Token Loss for Each Paragraph(Tyler)

In [170]:
tyler_analysis_df_w_no_context_first_paragraph = analyze_text(tyler_essay_first_paragraph)
display(analysis_df_w_no_context[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,Many,76.802307,increa,1.000000
1,▁people,1.230443,people,0.016021
2,▁believe,1.468063,believe,0.019115
3,▁that,0.070375,that,0.000916
4,▁there,4.952903,the,0.064489
...,...,...,...,...
125,▁including,15.139212,.,0.197119
126,▁their,1.828609,the,0.023809
127,▁ancestors,5.650682,families,0.073574
128,.,0.541190,.,0.007047


In [128]:
tyler_analysis_df_w_no_context_second_paragraph = analyze_text(tyler_essay_second_paragraph)
display(tyler_analysis_df_w_no_context_second_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,People,77.114807,increa,1.000000e+00
1,’,9.285842,often,1.204158e-01
2,s,0.000011,s,1.484041e-07
3,▁identities,12.982472,Republic,1.683525e-01
4,▁are,0.189558,are,2.458128e-03
...,...,...,...,...
632,▁he,0.099194,he,1.286310e-03
633,▁viewed,4.022060,saw,5.215678e-02
634,▁his,1.059572,his,1.374019e-02
635,▁story,5.849725,relationship,7.585735e-02


In [129]:
tyler_analysis_df_w_no_context_third_paragraph = analyze_text(tyler_essay_third_paragraph)
display(tyler_analysis_df_w_no_context_third_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,While,77.552307,increa,1.000000
1,▁some,5.756194,the,0.074223
2,▁believe,4.480068,people,0.057768
3,▁that,0.453788,that,0.005851
4,▁their,8.656082,the,0.111616
...,...,...,...,...
533,▁all,4.852514,because,0.062571
534,▁due,3.275755,thanks,0.042239
535,▁to,0.000131,to,0.000002
536,▁Diane,0.154984,Diane,0.001998


### Token Loss for Each Paragraph(Lydia)

### Token Loss for Each Paragraph(Chang)

In [ ]:
chang_analysis_df_w_no_context_first_paragraph = analyze_text(chang_essay_first_paragraph)
display(chang_analysis_df_w_no_context_first_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

In [ ]:
chang_analysis_df_w_no_context_second_paragraph = analyze_text(chang_essay_second_paragraph)
display(chang_analysis_df_w_no_context_second_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

In [ ]:
chang_analysis_df_w_no_context_third_paragraph = analyze_text(chang_essay_third_paragraph)
display(chang_analysis_df_w_no_context_third_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

# Compare the Two Options (Human Written vs ML Model Written) & Highlight Where Changes are Made

In [91]:
def highlight_revised_essay(essay, analysis_df):
    tokens = essay.split()  # Split the original essay into tokens (simple space-based tokenization might not perfectly match tokenizer behavior)
    revised_tokens = tokens.copy()  # Create a copy of the tokens for modification
    
    token_idx_to_replace = {}  # Dictionary to map original token index to most likely token
    
    # Iterate over the dataframe to replace tokens
    for index, row in analysis_df.iterrows():
        original_token = row['original_token']
        most_likely_token = row['most_likely_token']
        token_loss = row['token_loss']
        
        # Find the index of the token in the original tokens list
        # This assumes exact match; handling multiple occurrences or mismatches might require more sophisticated tracking
        try:
            token_index = tokens.index(original_token)
            if tokens[token_index] != most_likely_token:  # Check if the token is actually different
                token_idx_to_replace[token_index] = most_likely_token
        except ValueError:
            # If the token is not found due to differences in tokenization, ignore it
            continue
    
    # Replace and mark tokens in the copied list
    for idx, replacement in token_idx_to_replace.items():
        revised_tokens[idx] = f"[{replacement}]"  # Wrap the replacement in brackets to highlight it
    
    # Join the revised tokens back into a single string
    revised_essay = ' '.join(revised_tokens)
    return revised_essay

# Different Context Approaches
## Generate an Outline from Human Written Essay & Revise that Outline to Generate a New Paragraph

### Tyler's Essay

### Generate Essay Based on Model Generated Outline

#### First Paragraph

In [92]:
%%time
doc = f'''Please break this introduction paragraph into a detailed outline format focusing on the thesis statement and a brief introduction: {tyler_essay_first_paragraph} '''
model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>Please break this introduction paragraph into a detailed outline format focusing on the thesis statement and a brief introduction: Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors.  

**Thesis Statement:** This paper argues that the multiplicity of stories in the world reflects the richness and complexity of human experience, and that understanding the narrat

In [138]:
# Example code snippet for generating an introduction paragraph from an outline
doc = """
Please convert the following outline into a well-structured academic introduction paragraph. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.

Thesis Statement: This paper argues that the multiplicity of stories in the world reflects the richness and complexity of human experience, and that understanding the narratives of others can provide valuable insights into our own lives and the world around us.

Outline:
- Briefly discuss the common belief of a singular narrative in the world.
- Highlight the underestimation of the multiplicity of stories.
- Introduce the authors and films discussed in the paper.
- Explain how narratives shape our perspectives and understanding of the world.
- Discuss how narratives
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=200,
    do_sample=False,
    streamer=streamer
)

<bos>
Please convert the following outline into a well-structured academic introduction paragraph. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.

Thesis Statement: This paper argues that the multiplicity of stories in the world reflects the richness and complexity of human experience, and that understanding the narratives of others can provide valuable insights into our own lives and the world around us.

Outline:
- Briefly discuss the common belief of a singular narrative in the world.
- Highlight the underestimation of the multiplicity of stories.
- Introduce the authors and films discussed in the paper.
- Explain how narratives shape our perspectives and understanding of the world.
- Discuss how narratives
offer a deeper understanding of our own lives and the world around us.

**Introduction:**

The tapestry of human experience is woven with an intricate multiplicity of stories, each thread contributing to the ric

In [147]:
tyler_generated_first_paragraph_models_outline = '''
The tapestry of human experience is woven with an intricate multiplicity of stories, each thread contributing to the richness and complexity of our collective consciousness. While the singular narrative often holds prominence in popular discourse, neglecting the multiplicity of stories diminishes the depth and nuance of human understanding. This paper argues that the richness of human experience lies precisely in the multiplicity of narratives, and that understanding the narratives of others provides invaluable insights into our own lives and the world around us.'''
tyler_analysis_df_w_models_outline_first_paragraph = analyze_text(tyler_generated_first_paragraph_models_outline)
display(tyler_analysis_df_w_models_outline_first_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,\n,73.802307,increa,1.000000e+00
1,The,2.327003,**,3.153022e-02
2,▁tapestry,14.386633,provided,1.949347e-01
3,▁of,0.477380,of,6.468360e-03
4,▁human,1.958499,life,2.653710e-02
...,...,...,...,...
85,▁the,0.077418,the,1.048997e-03
86,▁world,0.259387,world,3.514614e-03
87,▁around,0.028808,around,3.903395e-04
88,▁us,0.000016,us,2.180607e-07


#### Second Paragraph

In [130]:
%%time
doc = f'''Please convert the following detailed discussion into a structured outline, emphasizing the main thesis about how personal choices shape identities and providing key examples from the texts: {tyler_essay_second_paragraph} '''
model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>Please convert the following detailed discussion into a structured outline, emphasizing the main thesis about how personal choices shape identities and providing key examples from the texts: People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we mainly see the story through the viewpoint of Sunja, but we will occasionally get to look through or see the thoughts of the people close to her. In the novel, it states, “If he did not marry her, she was a common slut who would be disgraced forever. The child would be another no-name bastard. Her mother’s boardinghouse would be contaminated by her shame”(Lee 49). Sunja made a big mistake by being with Hansu, and she paid the price. She was now pregnant, and the father of the child won’t be able to mar

In [145]:
# Example code snippet for generating the first body paragraph from an outline
doc = """
Compose a detailed body paragraph based on the theme that personal choices shape identities. Include:
1. The impact of Sunja's decision with Hansu on her and her family's reputation.
2. How H's time in prison made him a hardworking and responsible person.
3. The effect of Harry's biased view on his own identity and story.
Conclude with how these stories illustrate the link between personal choices and societal expectations.
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=400,  # Increase token limit to enhance detail
    do_sample=True,  # Enable sampling for more diverse responses
    streamer=streamer
)

<bos>
Compose a detailed body paragraph based on the theme that personal choices shape identities. Include:
1. The impact of Sunja's decision with Hansu on her and her family's reputation.
2. How H's time in prison made him a hardworking and responsible person.
3. The effect of Harry's biased view on his own identity and story.
Conclude with how these stories illustrate the link between personal choices and societal expectations.
-

Personal choices play a pivotal role in shaping identities, influencing how individuals view themselves and others, and shaping their relationships within broader societal contexts. One such example is Sunja's decision to leave Hansu and her family, which had a significant impact on both her personal and familial reputation. The stigma and embarrassment she faced led to a loss of opportunities, while the emotional turmoil inflicted on Hansu resulted in a strained relationship. However, this experience also strengthened Sunja's sense of independence and resi

In [148]:
tyler_generated_second_paragraph_models_outline = '''
Personal choices play a pivotal role in shaping identities, influencing how individuals view themselves and others, and shaping their relationships within broader societal contexts. One such example is Sunja's decision to leave Hansu and her family, which had a significant impact on both her personal and familial reputation. The stigma and embarrassment she faced led to a loss of opportunities, while the emotional turmoil inflicted on Hansu resulted in a strained relationship. However, this experience also strengthened Sunja's sense of independence and resilience, becoming instrumental in her future choices and navigating complex interpersonal dynamics.
H's time in prison not only transformed him into a hardworking and responsible individual but also instilled in him a sense of purpose and responsibility. The transformative power of adversity and the newfound skills learned behind bars created a foundation of resilience, strength, and newfound appreciation for life. His transformation served as a catalyst for personal growth and a determination to live a life of integrity and purpose upon his release.
Conversely, Harry's biased view of himself and his inability to confront his shortcomings inadvertently shaped his identity. His self-fulfilling prophecy served as a persistent obstacle to personal development and a failure to recognize his own flaws and limitations. Harry's inability to own up to his mistakes and blame others created a barrier to genuine intimacy and hindered his ability to forge healthy relationships.
In conclusion, these stories illustrate the potent connection between personal choices and societal expectations. Sunja's decision to leave Hansu demonstrated the power of personal autonomy and resilience, while H's transformation in prison showcased the transformative effects of adversity. Harry's biased self-perception and inability to confront his shortcomings served as a cautionary tale of how societal expectations and biases can perpetuate negative self-belief and limit personal growth.'''
tyler_analysis_df_w_models_outline_second_paragraph = analyze_text(tyler_generated_second_paragraph_models_outline)
display(tyler_analysis_df_w_models_outline_second_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,\n,73.802307,increa,1.000000
1,Personal,14.702003,**,0.199208
2,▁choices,12.514207,ize,0.169564
3,▁play,3.811835,and,0.051649
4,▁a,0.008368,a,0.000113
...,...,...,...,...
347,▁and,0.534257,and,0.007239
348,▁limit,2.858231,hinder,0.038728
349,▁personal,0.011539,personal,0.000156
350,▁growth,0.210311,growth,0.002850


#### Third Paragraph

In [131]:
%%time
doc = f'''Please transform the following detailed discussion into a structured outline, emphasizing the thesis on how external influences like family and societal contexts shape individual identities, and include key examples from the texts: {tyler_essay_third_paragraph} '''
model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>Please transform the following detailed discussion into a structured outline, emphasizing the thesis on how external influences like family and societal contexts shape individual identities, and include key examples from the texts: While some believe that their identity is solely based on their choices, people’s identities can be altered by the people around them and by their ancestors. When we start looking through the other lenses of people in the same situation, we can see that many times the expected outcome is different from reality. In the novel, Pachinko, it states, “‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'”(Lee 74). Isak knew the dilemma Sunja was in and he knew this was something that shouldn’t be taken lightly. After coming up with the idea of marrying Sunja and giving the child his last name, he had lifted the burden off Su

In [150]:
# Example code snippet for generating a body paragraph from an outline
doc = """
Compose a detailed body paragraph based on the thesis that external influences like family and societal contexts shape individual identities. Include the following key examples:
1. In 'Pachinko', Yangjin's decision reflects societal pressures and the harsh realities faced by unmarried mothers, as shown when she says, 'Of course it would be far better for them if she went away' and 'The child would have a terrible life here. You’d be saving my daughter’s life as well' (Lee 74).
2. In 'Homegoing', Ness's daily experience picking cotton under harsh conditions at the Alabama plantation, as stated: 'Every day, Ness picked cotton under the punishing eye of the southern sun. She had been at Thomas Allan Stockham’s Alabama plantation for three months' (Gyasi 70), illustrates the historical impact of slavery on individual identities.
3. Discuss the influence of personal and public narratives in shaping identities as explored in the documentary 'Stories We Tell'.
Conclude with how these examples collectively illustrate the profound impact of external influences on personal identity formation.
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=400,  # Increase token limit to allow for more comprehensive development
    do_sample=True,  # Enable sampling for more diverse responses
    streamer=streamer
)

<bos>
Compose a detailed body paragraph based on the thesis that external influences like family and societal contexts shape individual identities. Include the following key examples:
1. In 'Pachinko', Yangjin's decision reflects societal pressures and the harsh realities faced by unmarried mothers, as shown when she says, 'Of course it would be far better for them if she went away' and 'The child would have a terrible life here. You’d be saving my daughter’s life as well' (Lee 74).
2. In 'Homegoing', Ness's daily experience picking cotton under harsh conditions at the Alabama plantation, as stated: 'Every day, Ness picked cotton under the punishing eye of the southern sun. She had been at Thomas Allan Stockham’s Alabama plantation for three months' (Gyasi 70), illustrates the historical impact of slavery on individual identities.
3. Discuss the influence of personal and public narratives in shaping identities as explored in the documentary 'Stories We Tell'.
Conclude with how these ex

In [151]:
tyler_generated_third_paragraph_models_outline = '''
Throughout history, external influences such as family, societal contexts, and personal narratives have played a pivotal role in shaping individual identities. In literary works, historical narratives, and personal experiences, we witness how individuals' choices, experiences, and beliefs are shaped by societal pressures, historical legacies, and personal narratives. For instance, in "Pachinko," Yangjin's decision to end her life reflects the societal pressures and harsh realities faced by unmarried mothers. Similarly, in "Homegoing," Ness's daily labor on the Alabama plantation under oppressive conditions underscores the historical impact of slavery on individual identities. Moreover, personal narratives, such as those told in "Stories We Tell," reveal how individuals' experiences and perspectives shape their sense of self and belonging.
In conclusion, the external influences discussed in this paper collectively illustrate the profound impact of external factors on personal identity formation. Family, societal contexts, and personal narratives intertwine to shape our beliefs, values, and behaviors, ultimately contributing to the tapestry of our individual identities.
'''
tyler_analysis_df_w_models_outline_third_paragraph = analyze_text(tyler_generated_third_paragraph_models_outline)
display(tyler_analysis_df_w_models_outline_third_paragraph[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,\n,73.802307,increa,1.000000
1,Throughout,11.811378,**,0.160041
2,▁history,0.095976,history,0.001300
3,",",0.001863,",",0.000025
4,▁external,13.359329,there,0.181015
...,...,...,...,...
200,▁our,0.116734,our,0.001582
201,▁individual,0.319207,individual,0.004325
202,▁identities,1.825144,experiences,0.024730
203,.,0.000232,.,0.000003


### Lydia's Essay

### Chang's Essay

## Model Genereated Outline(Human Generated Outline Given)

### Tyler's Essay

In [95]:
# Example code snippet for generating an introduction paragraph from an outline
doc = f"""
Please convert the following outline into a well-structured academic introduction paragraph. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.{tyler_outline_first_paragraph}
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>
Please convert the following outline into a well-structured academic introduction paragraph. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.
Thesis: Authors and filmmaker use multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, demonstrating their theories on how history should be constructed.
Brief introduction of works and creators: Discusses Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.

This paper investigates how authors and filmmakers utilize multiple perspectives to depict the complexities of personal and ancestral choices, challenging traditional narratives and emphasizing the power of individual narratives to shape identity and history. By examining the works of Yaa Gyasi, Min Jin Lee, and Sarah Polley, this paper argues that personal an

In [96]:
tyler_generated_intro_paragraph_human_outline = '''This paper investigates how authors and filmmakers utilize multiple perspectives to depict the complexities of personal and ancestral choices, challenging traditional narratives and emphasizing the power of individual narratives to shape identity and history. By examining the works of Yaa Gyasi, Min Jin Lee, and Sarah Polley, this paper argues that personal and ancestral choices are not merely individual experiences but are also shaped by broader historical contexts, familial dynamics, and societal narratives.This paper seeks to demonstrate how authors and filmmakers employ multiple perspectives to illuminate the multifaceted nature of human experience, emphasizing the role of personal narratives in understanding the complexities of identity and history.'''
tyler_analysis_df_w_human_outline = analyze_text(tyler_generated_intro_paragraph_human_outline)
display(tyler_analysis_df_w_human_outline[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,This,72.989807,increa,1.000000
1,▁paper,10.340496,is,0.141670
2,▁investigates,2.858632,proposes,0.039165
3,▁how,4.138315,the,0.056697
4,▁authors,12.082332,the,0.165535
...,...,...,...,...
115,▁of,0.022485,of,0.000308
116,▁identity,0.638408,identity,0.008747
117,▁and,0.243705,and,0.003339
118,▁history,0.112638,history,0.001543


### Lydia's Essay

### Chang's Essay

## Generated Introduction Paragraph given Human Written Essay & Human Written Outline

### Tyler's Essay

In [97]:
# Example code snippet for generating an introduction paragraph from an outline
doc = f"""
Please revise my introduction paragraph following the outline I provided. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.{tyler_essay_first_paragraph}{tyler_outline_first_paragraph}
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>
Please revise my introduction paragraph following the outline I provided. The paragraph should integrate all the points cohesively, providing a clear and engaging introduction to the topic.Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. 
Thesis: Authors and filmmaker use multiple perspectives to illustrate how personal and ancestral choices shape individua

In [98]:
# Example code snippet for generating an introduction paragraph from an outline
doc = f"""
Based on these modifications, please revise my introduction paragraph {tyler_essay_first_paragraph}

Changes needed to be made:
- Add a thesis statement that explicitly connects the use of multiple perspectives to historical narratives.
- Integrate the specific works and creators into the context of the thesis.
- Clarifie the purpose of the introduction by establishing the connection between personal choices and historical narratives.

"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>
Based on these modifications, please revise my introduction paragraph Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. 

Changes needed to be made:
- Add a thesis statement that explicitly connects the use of multiple perspectives to historical narratives.
- Integrate the specific works and creators into the context of the thesis.
- Clarifie the purpose of 

In [99]:
tyler_generated_intro_paragraph_human_outline_and_essay = '''This paper investigates how authors and filmmakers utilize multiple perspectives to depict the complexities of personal and ancestral choices, challenging traditional narratives and emphasizing the power of individual narratives to shape identity and history. By examining the works of Yaa Gyasi, Min Jin Lee, and Sarah Polley, this paper argues that personal and ancestral choices are not merely individual experiences but are also shaped by broader historical contexts, familial dynamics, and societal narratives.This paper seeks to demonstrate how authors and filmmakers employ multiple perspectives to illuminate the multifaceted nature of human experience, emphasizing the role of personal narratives in understanding the complexities of identity and history.'''
tyler_analysis_df_w_human_outline_and_essay = analyze_text(tyler_generated_intro_paragraph_human_outline_and_essay)
display(tyler_analysis_df_w_human_outline_and_essay[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

,original_token,token_loss,most_likely_token,loss_ratio
0,This,72.989807,increa,1.000000
1,▁paper,10.340496,is,0.141670
2,▁investigates,2.858632,proposes,0.039165
3,▁how,4.138315,the,0.056697
4,▁authors,12.082332,the,0.165535
...,...,...,...,...
115,▁of,0.022485,of,0.000308
116,▁identity,0.638408,identity,0.008747
117,▁and,0.243705,and,0.003339
118,▁history,0.112638,history,0.001543


In [100]:
# Example code snippet for generating an introduction paragraph from an outline and existing essay content
doc = f"""
Generate a comprehensive introductory paragraph that synthesizes the following essay and outline into a cohesive narrative. The paragraph should integrate the perspectives and key points discussed, providing a clear and engaging introduction to the topic.

Essay:
'Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors.'

Outline:
Thesis: Authors and filmmakers use multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, demonstrating their theories on how history should be constructed.
Brief introduction of works and creators: Discusses Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.
"""

model_out = model.generate(
    **tokenizer(doc, return_tensors='pt').to(model.device),
    max_new_tokens=128,
    do_sample=False,
    streamer=streamer
)

<bos>
Generate a comprehensive introductory paragraph that synthesizes the following essay and outline into a cohesive narrative. The paragraph should integrate the perspectives and key points discussed, providing a clear and engaging introduction to the topic.

Essay:
'Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors.'

Outline:
Thesis: Authors and filmmakers use

In [101]:
tyler_first_paragraph_generated_w_outline_and_essay = '''
The exploration of multiple perspectives in historical narratives offers a powerful lens through which to understand the complex interplay between individual choices, historical context, and the collective experiences of those who lived through them. By examining the narratives of Homegoing, Pachinko, and Stories We Tell, we can see how the choices made by individuals and their communities shape the narratives of their lives, and the people around them, including their ancestors, become integral components of the historical narrative.<eos>
'''

In [102]:
tyler_first_paragraph_generated_w_outline_and_essay_2 = '''In the tapestry of human existence, where narratives intertwine, the question of how individual experiences are shaped by broader contexts and the choices we make has always fascinated writers and thinkers. While the notion of a singular, overarching story may hold merit, the complexities of human life suggest that narratives are not merely linear journeys but multifaceted tapestries woven from the threads of personal choices, ancestral legacies, and the collective experiences that shape our collective consciousness. Authors and filmmakers like Yaa Gyasi, Min Jin Lee, and Sarah Polley, through their'''

### Lydia's Essay

### Chang's Essay

# Essay Results
## Generated Introduction Paragraph Given Desired Human Outline

## Generated Introduction Paragraph Given Model's Outline(From Essay)
### Tyler's Essay

In [171]:
tyler_revised_essay_with_highlights_first_paragraph = highlight_revised_essay(tyler_essay_first_paragraph, tyler_analysis_df_w_models_outline_first_paragraph)
tyler_revised_essay_with_highlights_second_paragraph = highlight_revised_essay(tyler_essay_second_paragraph, tyler_analysis_df_w_models_outline_second_paragraph)
tyler_revised_essay_with_highlights_third_paragraph = highlight_revised_essay(tyler_essay_third_paragraph, tyler_analysis_df_w_models_outline_third_paragraph)
print("Original Essay:\n", tyler_essay_first_paragraph + "\n\n" + tyler_essay_second_paragraph + "\n\n" + tyler_essay_third_paragraph)
print("\n\n")
print("\nRevised Introduction Based on Most Likely Tokens(With Human's Outline):\n", tyler_revised_essay_with_highlights_first_paragraph + "\n\n" + tyler_revised_essay_with_highlights_second_paragraph + "\n\n" + tyler_revised_essay_with_highlights_third_paragraph)

Original Essay:
 Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. 

People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we 

## Genereated Introduction Paragraph Given Human Essay & Outline

In [172]:
tyler_revised_essay_with_human_outline_and_essay = highlight_revised_essay(tyler_essay_first_paragraph, tyler_analysis_df_w_human_outline_and_essay)
print("Original Introduction Paragraph:\n", tyler_essay_first_paragraph)
print("\nRevised Introduction Based on Most Likely Tokens(With Models Outline):\n", tyler_revised_essay_with_human_outline_and_essay)

Original Introduction Paragraph:
 Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. 

Revised Introduction Based on Most Likely Tokens(With Models Outline):
 Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small s

## Generated Introduction Given Model's Outline(From Prompt/Generic)

## Generated Introduction Paragraph Based On Most_likely_token(NO CONTEXT)

In [173]:
# Apply the revision function to the original essay
tyler_revised_essay = highlight_revised_essay(tyler_essay_first_paragraph, tyler_analysis_df_w_no_context)
print("Original Introduction Paragraph:\n", tyler_essay_first_paragraph)
print("\nRevised Introduction Based on Most Likely Tokens(NO CONTEXT):\n", tyler_revised_essay)

Original Introduction Paragraph:
 Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. 

Revised Introduction Based on Most Likely Tokens(NO CONTEXT):
 [ increa] people believe that there is only one story [ Alone] on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small si

# Results
Include quantitative(tables, plots) and qualitiative(examples) results, including comparisons with similar work if applicable.

# Implications
Discuss the social and ethical implications of using the technologies you've chosen for your project